# 12. Cluster Analysis


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec
from sklearn.cluster import KMeans


In [3]:
%run "01.Recommendation_Loading.ipynb"

################  Recommendations (all data, positive recommendations only) ################
Users: 12,663,134
Games: 37,419
Number of users 10,000
First five users id [11187923 12792392 11703922  9531989 11022607]
################  Recommendations (Sample) ################
Minimum number of recommendations to be included in the sample: 20
Number of randomly selected users: 10000
Users: 9,690
Games: 21,487
################ User-Game Matrix ################
Number of Rows: 9,691
Numbeer of Columns: 21,488
Number of stored values: 369,103


In [4]:
%run "04.Games_Exploration.ipynb"


There are 50872 distinct games
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50872 entries, 0 to 50871
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   app_id       50872 non-null  int64 
 1   description  50872 non-null  object
 2   tags         50872 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB
There are 50872 distinct games
There are 441 tags
The top 10 tags with their respective count are: [('Indie', 27957), ('Singleplayer', 22566), ('Action', 21897), ('Adventure', 20183), ('Casual', 17844)]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49628 entries, 10 to 2515460
Columns: 441 entries, 1980s to eSports
dtypes: int8(441)
memory usage: 21.3 MB


In [5]:
recommendations

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,user_id_categorical,app_id_categorical
32197893,761620,6,0,2021-08-24,1,56.8,2881,32197893,0,10708
35310498,274520,4,0,2021-08-31,1,146.5,2881,35310498,0,2195
25716249,244450,0,0,2021-03-19,1,248.2,2881,25716249,0,1599
35309426,324800,28,4,2021-08-24,1,122.4,2881,35309426,0,3282
32301016,34870,0,2,2021-09-05,1,135.0,2881,32301016,0,569
...,...,...,...,...,...,...,...,...,...,...
35966762,311340,0,0,2019-06-29,1,10.5,14305218,35966762,9689,2955
24564236,359550,0,0,2016-11-11,1,65.2,14305218,24564236,9689,4108
20181065,594650,3,0,2018-08-11,1,418.6,14305218,20181065,9689,8442
36808721,618510,0,0,2018-02-09,1,0.3,14305218,36808721,9689,8818


In [6]:
matrix_games_tags = matrix_games_tags.reset_index()
matrix_games_tags

tags,app_id,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,...,Well-Written,Werewolves,Western,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,eSports
0,10,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,20,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
3,40,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49623,2478270,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
49624,2488153,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49625,2510770,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49626,2515240,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
games_to_keep = recommendations.app_id.unique()

In [8]:
matrix_games_tags = matrix_games_tags[matrix_games_tags["app_id"].isin(games_to_keep)]
matrix_games_tags

tags,app_id,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,...,Well-Written,Werewolves,Western,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,eSports
0,10,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,20,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
3,40,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48311,2235760,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
48312,2235780,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
48335,2237980,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
48359,2240530,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
games_idx = pd.read_csv('matrix/games_idx.csv')
games_idx.head()


,app_id,app_id_categorical
0,761620,10708
1,274520,2195
2,244450,1599
3,324800,3282
4,34870,569


In [10]:
matrix_games_tags = pd.merge(games_idx, matrix_games_tags, left_on="app_id", right_on="app_id").drop(columns="app_id")
matrix_games_tags

,app_id_categorical,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,...,Well-Written,Werewolves,Western,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,eSports
0,10708,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1599,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,3282,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,569,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20908,14167,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20909,6158,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20910,16845,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20911,9653,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
recommendations

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,user_id_categorical,app_id_categorical
32197893,761620,6,0,2021-08-24,1,56.8,2881,32197893,0,10708
35310498,274520,4,0,2021-08-31,1,146.5,2881,35310498,0,2195
25716249,244450,0,0,2021-03-19,1,248.2,2881,25716249,0,1599
35309426,324800,28,4,2021-08-24,1,122.4,2881,35309426,0,3282
32301016,34870,0,2,2021-09-05,1,135.0,2881,32301016,0,569
...,...,...,...,...,...,...,...,...,...,...
35966762,311340,0,0,2019-06-29,1,10.5,14305218,35966762,9689,2955
24564236,359550,0,0,2016-11-11,1,65.2,14305218,24564236,9689,4108
20181065,594650,3,0,2018-08-11,1,418.6,14305218,20181065,9689,8442
36808721,618510,0,0,2018-02-09,1,0.3,14305218,36808721,9689,8818


In [12]:
matrix_games_tags = pd.merge(recommendations, matrix_games_tags, left_on="app_id_categorical", right_on="app_id_categorical")
matrix_games_tags

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,user_id_categorical,app_id_categorical,...,Well-Written,Werewolves,Western,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,eSports
0,761620,6,0,2021-08-24,1,56.8,2881,32197893,0,10708,...,0,0,0,0,0,0,0,0,0,0
1,761620,0,0,2021-06-14,1,13.6,52992,29890144,42,10708,...,0,0,0,0,0,0,0,0,0,0
2,761620,0,0,2022-03-20,1,6.0,68287,29972752,48,10708,...,0,0,0,0,0,0,0,0,0,0
3,761620,0,0,2020-12-05,1,27.0,447120,40757307,249,10708,...,0,0,0,0,0,0,0,0,0,0
4,761620,0,0,2022-03-22,1,18.0,1910105,32997992,930,10708,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253932,1063530,6,0,2019-11-20,1,0.2,14268141,28568574,9664,14167,...,0,0,0,0,0,0,0,0,0,0
253933,456180,4,0,2019-06-27,1,10.0,14268141,40048755,9664,6158,...,0,0,0,0,0,0,0,0,0,0
253934,1318800,4,0,2021-10-04,1,1.2,14284627,27806881,9674,16845,...,0,0,0,0,0,0,0,0,0,0
253935,681530,7,0,2019-01-13,1,16.2,14284627,32765232,9674,9653,...,0,0,0,0,0,0,0,0,0,0


In [13]:
matrix_games_tags = matrix_games_tags.groupby("user_id_categorical").max()


In [14]:
matrix_games_tags

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,app_id_categorical,1980s,...,Well-Written,Werewolves,Western,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,eSports
user_id_categorical,,,,,,,,,,,,,,,,,,,,,
0,1461680,124,47,2022-05-19,1,968.1,2881,40897360,17982,1,...,1,0,1,0,0,1,1,0,1,1
1,2102020,228,35,2022-12-11,1,64.3,3737,40002727,21274,0,...,0,0,0,0,0,0,0,0,1,0
2,1889620,4,4,2022-12-26,1,213.4,5488,40410867,20498,1,...,0,1,0,0,0,0,0,0,1,1
3,1456230,39,35,2022-01-21,1,35.5,6162,40134652,17933,0,...,0,0,0,1,0,0,1,0,1,0
4,640820,76,21,2020-02-27,1,121.3,6566,35889705,9103,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9685,1417010,17,11,2022-06-27,1,85.9,14302978,40037627,17624,0,...,0,0,0,0,0,0,0,0,1,0
9686,1549250,40,3,2022-12-06,1,60.9,14303060,39371119,18627,1,...,0,0,0,0,0,0,1,0,1,0
9687,1921560,21,12,2022-12-30,1,116.0,14303095,40793968,20665,1,...,0,0,0,0,1,0,0,0,0,0


In [15]:
columns_to_drop = list(recommendations.columns)
columns_to_drop.remove("user_id_categorical")
columns_to_drop

['app_id',
 'helpful',
 'funny',
 'date',
 'is_recommended',
 'hours',
 'user_id',
 'review_id',
 'app_id_categorical']

In [16]:
matrix_games_tags = matrix_games_tags.drop(columns=columns_to_drop)
matrix_games_tags

,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,3D Platformer,...,Well-Written,Werewolves,Western,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,eSports
user_id_categorical,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,0,1,0,1,0,1,...,1,0,1,0,0,1,1,0,1,1
1,0,1,0,1,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
2,1,1,0,1,0,1,0,1,0,1,...,0,1,0,0,0,0,0,0,1,1
3,0,1,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,1,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9685,0,1,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
9686,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
9687,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [17]:
matrix_games_tags.T

user_id_categorical,0,1,2,3,4,5,6,7,8,9,...,9680,9681,9682,9683,9684,9685,9686,9687,9688,9689
1980s,1,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1,1,0,1
1990's,1,1,1,1,0,0,0,0,1,1,...,0,0,1,1,0,1,0,0,1,1
2.5D,1,0,0,0,0,1,1,1,0,0,...,0,0,1,1,0,1,0,0,1,1
2D,1,1,1,1,1,1,1,1,1,1,...,0,1,0,1,1,1,0,1,1,1
2D Fighter,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
World War I,1,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
World War II,1,0,0,1,1,0,1,0,0,1,...,0,0,0,1,0,0,1,0,0,1
Wrestling,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Zombies,1,1,1,1,1,1,0,0,0,1,...,0,1,0,1,0,1,1,0,0,1


## Principal Component Analysis

In [18]:
# Standardize the data
scaler = StandardScaler()
game_user_matrix_standardized = scaler.fit_transform(matrix_games_tags.T)

In [19]:
# Specify the number of components (dimensions) you want after reduction
num_components = 10

# Apply PCA
pca = PCA(n_components=num_components)
game_tags_reduced = pca.fit_transform(game_user_matrix_standardized)

# Print the reduced dimensions for each game tag
print("Reduced Game Tags Dimensions:\n", game_tags_reduced)

game_tags_reduced.shape

Reduced Game Tags Dimensions:
 [[-3.65114737e-01 -5.86890608e+00 -6.25913790e+00 ...  2.16338864e+01
  -2.60179486e+00  6.05878436e+00]
 [ 2.52375022e+01 -7.61567677e+00 -8.39622108e+00 ...  3.16660128e+01
  -8.25328408e+00  1.06807858e+01]
 [ 1.93881146e+01 -2.31011757e+01  7.11278539e+00 ... -1.39085015e+01
  -3.08464657e+00 -5.55414295e+00]
 ...
 [-6.02901847e+01  3.80241530e+00 -5.28442834e+00 ... -9.72938202e-01
  -3.60110659e+00 -1.42336727e+00]
 [ 8.09722010e+01  2.17626794e+01 -2.45073709e+01 ...  1.15262766e+01
   2.83670405e+00  1.93511196e+01]
 [-4.94094663e+00  1.83485545e+01 -1.16851077e+01 ...  9.34926753e+00
  -1.66033681e-02 -1.69173002e+01]]


(441, 10)

In [20]:
comp_df = pd.DataFrame(data=game_tags_reduced, index=matrix_games_tags.T.index)
comp_df

,0,1,2,3,4,5,6,7,8,9
1980s,-0.4,-5.9,-6.3,-16.8,23.0,-11.5,2.4,21.6,-2.6,6.1
1990's,25.2,-7.6,-8.4,-14.8,25.1,-13.3,-4.2,31.7,-8.3,10.7
2.5D,19.4,-23.1,7.1,-0.2,18.4,0.5,8.0,-13.9,-3.1,-5.6
2D,132.9,-8.5,17.5,-11.6,-11.0,6.8,-6.7,1.9,-18.5,-0.0
2D Fighter,-15.5,-4.6,-3.7,-12.6,19.4,22.2,25.2,-6.3,-10.0,-6.7
...,...,...,...,...,...,...,...,...,...,...
World War I,-40.8,11.6,-6.8,0.3,-7.6,-4.9,5.2,8.2,-4.5,-1.0
World War II,11.5,29.8,-14.7,13.4,2.3,-16.9,21.1,15.2,3.0,2.9
Wrestling,-60.3,3.8,-5.3,-1.4,-3.2,6.3,-4.4,-1.0,-3.6,-1.4
Zombies,81.0,21.8,-24.5,-3.7,16.3,-6.6,-4.8,11.5,2.8,19.4


In [21]:
for comp in comp_df.columns:
    print("Component:", comp)
    print("Top Game Tags in this component:")
    print(comp_df[comp].sort_values(ascending=False)[:10])
    print("")

Component: 0
Top Game Tags in this component:
Singleplayer       157.4
Action             157.2
Adventure          156.9
Atmospheric        155.5
Multiplayer        155.4
Indie              153.9
Great Soundtrack   152.2
Story Rich         150.0
Open World         149.5
Co-op              148.0
Name: 0, dtype: float64

Component: 1
Top Game Tags in this component:
Military                54.0
Massively Multiplayer   52.4
Team-Based              51.5
War                     46.9
PvP                     45.1
Moddable                43.9
Tactical                43.5
Realistic               40.4
RTS                     39.8
Base Building           37.4
Name: 1, dtype: float64

Component: 2
Top Game Tags in this component:
Roguelike               55.4
Roguelite               53.5
Procedural Generation   48.0
Turn-Based Strategy     47.6
Management              46.3
Turn-Based              46.1
Resource Management     45.6
Building                44.8
Turn-Based Tactics      44.7
Dungeon Cra

Let's assign names to each component

In [22]:
comp_df.columns = ["Adventure", "War Muliplayer", "Management", "Strategy", "Simulation",
                  "RPG","Romantic", "Driving", "Realistic", "Horror"]

In [23]:
comp_df.head()

,Adventure,War Muliplayer,Management,Strategy,Simulation,RPG,Romantic,Driving,Realistic,Horror
1980s,-0.4,-5.9,-6.3,-16.8,23.0,-11.5,2.4,21.6,-2.6,6.1
1990's,25.2,-7.6,-8.4,-14.8,25.1,-13.3,-4.2,31.7,-8.3,10.7
2.5D,19.4,-23.1,7.1,-0.2,18.4,0.5,8.0,-13.9,-3.1,-5.6
2D,132.9,-8.5,17.5,-11.6,-11.0,6.8,-6.7,1.9,-18.5,-0.0
2D Fighter,-15.5,-4.6,-3.7,-12.6,19.4,22.2,25.2,-6.3,-10.0,-6.7


In [24]:
comp_df.columns

Index(['Adventure', 'War Muliplayer', 'Management', 'Strategy', 'Simulation',
       'RPG', 'Romantic', 'Driving', 'Realistic', 'Horror'],
      dtype='object')

In [25]:
comp_df.sort_values(by="War Muliplayer", ascending=False)

,Adventure,War Muliplayer,Management,Strategy,Simulation,RPG,Romantic,Driving,Realistic,Horror
Military,45.7,54.0,-13.9,10.7,6.1,-20.9,27.3,25.1,4.2,-9.5
Massively Multiplayer,51.5,52.4,-6.9,-24.7,-8.4,1.8,18.3,9.8,20.6,-5.1
Team-Based,46.1,51.5,-12.7,-16.9,9.2,0.7,25.0,20.0,17.4,-22.8
War,63.6,46.9,-9.1,25.3,9.1,-23.9,27.2,13.4,1.9,-7.4
PvP,93.9,45.1,-0.4,-20.2,-0.0,11.2,18.3,11.9,20.3,-22.7
...,...,...,...,...,...,...,...,...,...,...
Surreal,43.1,-45.7,-2.8,-13.7,6.9,-17.7,-14.0,17.2,3.9,-2.8
Interactive Fiction,7.2,-46.2,3.6,6.4,-23.2,-11.1,9.5,12.9,11.2,-5.3
Emotional,34.2,-47.6,-3.4,14.4,-16.9,-13.4,10.3,1.6,5.5,-10.5
Mystery,67.3,-48.5,-12.1,8.9,-19.6,-16.9,-4.9,7.4,12.2,-17.6


In [26]:
# Function to assign each tag to the component with the highest loading
def assign_to_component(row):
    return row.idxmax()

# Apply the function to each row (tag)
comp_df['Assigned_Component'] = comp_df.apply(assign_to_component, axis=1)

# Display the resulting DataFrame with assigned components
comp_df

,Adventure,War Muliplayer,Management,Strategy,Simulation,RPG,Romantic,Driving,Realistic,Horror,Assigned_Component
1980s,-0.4,-5.9,-6.3,-16.8,23.0,-11.5,2.4,21.6,-2.6,6.1,Simulation
1990's,25.2,-7.6,-8.4,-14.8,25.1,-13.3,-4.2,31.7,-8.3,10.7,Driving
2.5D,19.4,-23.1,7.1,-0.2,18.4,0.5,8.0,-13.9,-3.1,-5.6,Adventure
2D,132.9,-8.5,17.5,-11.6,-11.0,6.8,-6.7,1.9,-18.5,-0.0,Adventure
2D Fighter,-15.5,-4.6,-3.7,-12.6,19.4,22.2,25.2,-6.3,-10.0,-6.7,Romantic
...,...,...,...,...,...,...,...,...,...,...,...
World War I,-40.8,11.6,-6.8,0.3,-7.6,-4.9,5.2,8.2,-4.5,-1.0,War Muliplayer
World War II,11.5,29.8,-14.7,13.4,2.3,-16.9,21.1,15.2,3.0,2.9,War Muliplayer
Wrestling,-60.3,3.8,-5.3,-1.4,-3.2,6.3,-4.4,-1.0,-3.6,-1.4,RPG
Zombies,81.0,21.8,-24.5,-3.7,16.3,-6.6,-4.8,11.5,2.8,19.4,Adventure


In [27]:
matrix_games_tags_comp =pd.merge(matrix_games_tags.T,comp_df["Assigned_Component"], left_index=True, right_index=True)\
            .groupby("Assigned_Component").max().T
matrix_games_tags_comp.head()

Assigned_Component,Adventure,Driving,Horror,Management,RPG,Realistic,Romantic,Simulation,Strategy,War Muliplayer
0,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1
4,1,1,0,1,1,0,1,1,1,1


In [28]:
matrix_games_tags_comp = matrix_games_tags_comp.rename_axis('', axis=1)
matrix_games_tags_comp

,Adventure,Driving,Horror,Management,RPG,Realistic,Romantic,Simulation,Strategy,War Muliplayer
0,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1
4,1,1,0,1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
9685,1,1,1,1,1,1,1,1,1,1
9686,1,0,0,1,1,1,1,1,1,1
9687,1,1,1,1,1,1,1,1,1,1
9688,1,1,1,1,0,1,1,1,1,1


## Hierarchical Cluster Analysis

Grouping users of similar preference

In [29]:
# Compute pairwise distances
pairwise_distances = pdist(matrix_games_tags_comp)

# Perform hierarchical clustering
linkage_matrix = linkage(pairwise_distances, method='ward', metric="Euclidean")

linkage_matrix

array([[0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 2.00000000e+00],
       [2.00000000e+00, 9.69000000e+03, 0.00000000e+00, 3.00000000e+00],
       [3.00000000e+00, 9.69100000e+03, 0.00000000e+00, 4.00000000e+00],
       ...,
       [1.93660000e+04, 1.93750000e+04, 3.43780959e+01, 3.07900000e+03],
       [1.93690000e+04, 1.93760000e+04, 4.03486240e+01, 4.52300000e+03],
       [1.93740000e+04, 1.93770000e+04, 5.15921414e+01, 9.69000000e+03]])

In [30]:
linkage_matrix.shape

(9689, 4)

In [ ]:
linkage_matrix

## Applying Word2Vec to Gaems-Tag (An exploration)

In [ ]:
# !pip install gensim

In [ ]:
game_tags = list(matrix_games_tags.columns)

# Load Word2Vec model
sentences = [tag.split() for tag in game_tags]
model = Word2Vec(sentences, vector_size=20, window=5, min_count=1, workers=4)

# Get vectors for each tag, handling out-of-vocabulary cases
tag_vectors = [model.wv[tag] if tag in model.wv else np.zeros(model.vector_size) for tag in game_tags]

# Standardize the data
scaler = StandardScaler()
tag_vectors_standardized = scaler.fit_transform(tag_vectors)

# Specify the number of clusters
num_clusters = 10

# Apply K-means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
tag_clusters = kmeans.fit_predict(tag_vectors_standardized)

# Print the clusters for each tag
for tag, cluster in zip(game_tags, tag_clusters):
    print(f"{tag}: Cluster {cluster}")

tag_cluster_df = pd.DataFrame([tag_clusters], columns=game_tags).T\
                .reset_index()\
                .rename({0:"Cluster", "index":"Tag"}, axis=1)
tag_cluster_df.head()

1980s: Cluster 4
1990's: Cluster 7
2.5D: Cluster 9
2D: Cluster 6
2D Fighter: Cluster 2
2D Platformer: Cluster 2
360 Video: Cluster 2
3D: Cluster 4
3D Fighter: Cluster 2
3D Platformer: Cluster 2
3D Vision: Cluster 2
4 Player Local: Cluster 2
4X: Cluster 2
6DOF: Cluster 7
8-bit Music: Cluster 2
ATV: Cluster 2
Abstract: Cluster 5
Action: Cluster 4
Action RPG: Cluster 2
Action RTS: Cluster 2
Action Roguelike: Cluster 2
Action-Adventure: Cluster 8
Addictive: Cluster 5
Adventure: Cluster 8
Agriculture: Cluster 4
Aliens: Cluster 5
Alternate History: Cluster 2
Ambient: Cluster 3
America: Cluster 2
Animation & Modeling: Cluster 2
Anime: Cluster 9
Arcade: Cluster 2
Archery: Cluster 3
Arena Shooter: Cluster 2
Artificial Intelligence: Cluster 2
Assassin: Cluster 2
Asymmetric VR: Cluster 2
Asynchronous Multiplayer: Cluster 2
Atmospheric: Cluster 8
Audio Production: Cluster 2
Auto Battler: Cluster 2
Automation: Cluster 9
Automobile Sim: Cluster 2
BMX: Cluster 0
Base Building: Cluster 2
Baseball: Clu

In [ ]:
tag_cluster_df[tag_cluster_df["Cluster"]==0]

,Tag,Cluster
43,BMX,0
86,Combat,0
99,Crime,0
110,Deckbuilding,0
117,Diplomacy,0
134,Episodic,0
150,First-Person,0
187,Hunting,0
235,Minimalist,0
237,Mod,0


In [ ]:
# matrix_games_tags[matrix_games_tags["user_id_categorical"]==0]